# Model crystalographic structures using Modeller

- This notebook aimed to model missing loops inside the PDB structures of the target protein.

In [1]:
from pathlib import Path
from glob import glob
from prody import parsePDB
import sys
sys.path.append('../..')
from helper_modules.run_modeller import *
from helper_modules.get_pdb_ids_from_uniport import *

### Inputs
- Define some basic properties of the target protein.


In [2]:
prot_name       = 'egfr'
uniprot_id      = 'P00533'
ref_struc_id    = '2rgp'

In [3]:
seq_prot_full = get_seq_from_uniprot(uniprot_id)
print(seq_prot_full)
print(f'\nThere are {len(seq_prot_full)} residues.')

MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLEGCPTNGPKIPSIATGMVGALLLLLVVALGIGLFMRRRHIVRKRTLRRLLQERELVEPLTPSGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNFYRA

<h4 style='color: black; background-color: #F9E5AB; padding: 5px;'>
    Important!
</h4>

- <mark>NOTE:</mark> We will used a subsequece of the EGFR and HSP90 protein, as we are only interested in modeling the protein's active site.
- For the EGFR and HSP90 we will use a pdb id as a reference to get only the region containing the active site.

## Keep only the binding site region

- For this protein we only will use the region comprising the binding site.
- Only the subsequence going from positions 685 to 950 will be considered.

In [4]:
# Get the reference structure and its sequence and residue positions
seq_cry, positions_cry = get_structure_sequence(ref_struc_id)
print(len(seq_cry))
smin = positions_cry[0]
smax = positions_cry[-1]

print(smin, smax)

seq_prot = seq_cry
seq_prot

284
702 1016


'ALLRILKETEFKKIKVLGSGAFGTVYKGLWIPVKIPVAIKELRKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMNFYRALMDVVDADEY'

<h4 style='color: black; background-color: #F9E5AB; padding: 5px;'>
    Important!
</h4>

- For EGFR we will hardcode the starting and ending positions of the substructure to be modeled. This allow us to omit terminal regions not presented in all of the structures, which will require a template structure to be modeled.

In [5]:
# Hardcoded
start_res = 675
end_res = 950
smin = start_res
smax = end_res

print(smin, smax)

seq_prot = seq_prot_full[smin:smax]
seq_prot

685 950


'KRTLRRLLQERELVEPLTPSGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKC'

In [6]:
pdbid = '1m14'
stc_prot = parsePDB(pdbid)
stc_prot = stc_prot.select('not nonstdaa and resid > 0') 
seq_cry = stc_prot.select('ca').getSequence()
print('>>>', stc_prot.getResnames())
print(seq_cry)

>>> ['GLY' 'GLY' 'GLY' ... 'HOH' 'HOH' 'HOH']
GEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDDEEDMDDVVDADEYLIPQ


## Start the Modelling process
### Define the input and output directories

In [7]:
OUT_MAIN   = './pdb_structures'

# Get the list of input files
INPUT_DIR = f'{OUT_MAIN}/pdb_chains'
input_files = sorted(glob(f'{INPUT_DIR}/*pdb'))

# Define the output directory
OUTPUT_DIR = f'{OUT_MAIN}/pdb_modeled'
Path(OUTPUT_DIR).mkdir(parents = True, exist_ok = True)

In [9]:
# Model all molecules
for pdb_file in input_files:
    # Load the pdb file
    pdb_chain = parsePDB(pdb_file)
    
    # Run modeller
    run_modeller(
                 pdb_file = pdb_file, 
                 seq_prot = seq_prot, 
                 output_dir = OUTPUT_DIR, 
                 keep_original_resnum = False,
                 num_res_window = 0, 
                 max_var_iterations = 500, 
                 repeat_optimization = 2,
                 chid = 'A',
                 verbose = True,
                 start_position = smin,
                 end_position = smax
                )

>>> ['GLU' 'GLU' 'GLU' ... 'ASP' 'ASP' 'ASP']
./pdb_structures/pdb_chains/1m14_A.pdb
ETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARD
***** 266
Modelling protein 1m14_A
ETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARD KRTLRRLLQERELVEPLTPSGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKC
*****
Alignment(seqA='---------------------------------ETEFKKIKVLGSGAF

Finished!